In [6]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

# 웹드라이버 실행 (헤드리스 X)
driver = webdriver.Chrome()

# 구글 뉴스 목록 첫 페이지 URL
base_url = "https://www.google.com/search?sca_esv=441e37307c978c03&biw=1728&bih=888&q=%ED%98%84%EB%8C%80%EC%A4%91%EA%B3%B5%EC%97%85&tbm=nws&source=lnms"
driver.get(base_url)

# 페이지 로딩 대기
wait = WebDriverWait(driver, 10)

# 뉴스 데이터 저장 리스트
news_data = []
page = 1

while True:
    print(f"\n🔹 {page}페이지 크롤링 시작...")

    # 페이지 로딩 기다리기 (StaleElementReferenceException 방지)
    time.sleep(3)

    # 뉴스 컨테이너 가져오기 (반복문 내에서 요소를 매번 새로 찾음)
    try:
        news_list = driver.find_elements(By.CLASS_NAME, "SoaBEf")
        print(f"총 {len(news_list)}개의 뉴스를 찾았습니다.")
    except:
        print("❌ 뉴스 요소를 찾을 수 없습니다. 종료합니다.")
        break

    # 뉴스 목록에서 데이터 추출
    for i, news in enumerate(news_list):
        try:
            title = news.find_element(By.CLASS_NAME, "n0jPhd").text  # 제목
            summary = news.find_element(By.CLASS_NAME, "GI74Re").text  # 요약
            publisher = news.find_element(By.CLASS_NAME, "MgUUmf").text  # 출판사
            date = news.find_element(By.CLASS_NAME, "OSrXXb").text  # 날짜
            article_url = news.find_element(By.CLASS_NAME, "WlydOe").get_attribute("href")  # 기사 링크

            print(f"\n[{(page-1)*10 + i+1}] {title} ({date})")
            print(f"출판사: {publisher}")
            print(f"기사 링크: {article_url}")

            # ✅ 기사 본문 크롤링
            driver.execute_script("window.open('');")  # 새 탭 열기
            driver.switch_to.window(driver.window_handles[1])  # 새 탭으로 전환
            driver.get(article_url)  # 기사 페이지 열기
            time.sleep(3)  # 페이지 로딩 대기

            try:
                article_body = driver.find_element(By.CSS_SELECTOR, '[itemprop="articleBody"]').text
            except NoSuchElementException:
                article_body = "본문 수집 실패"

            print(f"본문 미리보기: {article_body[:100]}...")  # 본문 일부 출력

            # ✅ 새 탭 닫고 원래 탭으로 이동
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(3)

            # 데이터 저장
            news_data.append({
                "제목": title,
                "요약": summary,
                "출판사": publisher,
                "날짜": date,
                "기사 링크": article_url,
                "본문": article_body
            })

        except StaleElementReferenceException:
            print("⚠️ StaleElementReferenceException 발생! 다시 시도합니다.")
            time.sleep(2)
            continue
        except Exception as e:
            print(f"❌ 오류 발생: {e}")

    # 다음 페이지 버튼 클릭
    try:
        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="botstuff"]/div/div[3]/table/tbody/tr/td[{page+2}]/a')))
        next_button.click()
        time.sleep(3)
        page += 1  # 페이지 카운트 증가
    except NoSuchElementException:
        print("\n✅ 더 이상 다음 페이지가 없습니다. 크롤링 종료.")
        break

# 데이터프레임 생성
df = pd.DataFrame(news_data)

# CSV 파일로 저장
df.to_csv("google_news_hyundai_all_pages.csv", encoding="utf-8-sig", index=False)
print("\n✅ 데이터 수집 완료! 'google_news_hyundai_all_pages.csv' 파일로 저장되었습니다.")

# 드라이버 종료
driver.quit()



🔹 1페이지 크롤링 시작...
총 10개의 뉴스를 찾았습니다.

[1] 동서발전, HD현대중공업과 '탄소 에너지 공동 개발'...수소·암모니아 엔진 및 청정수소 기술 개발 협력 (2일 전)
출판사: 글로벌이코노믹
기사 링크: https://www.g-enews.com/article/Real-Estate/2025/02/202502071216274491a9fc143920_1
본문 미리보기: 이미지 확대보기
권명호 한국동서발전 사장(오른쪽)과 한주석 HD현대중공업 엔진기계사업대표가 지난 6일 HD현대중공업 본사에서 업무협약을 체결하고 기념사진을 찍고 있다. 사진=한국동...

[2] [단독] HD현대·한화, K해양방산 ‘원팀’ 합의 (2일 전)
출판사: 매일경제
기사 링크: https://www.mk.co.kr/news/business/11234873
본문 미리보기: 수출마케팅 주도권 분담 ‘큰그림’ 합의
향후 5년간 300억달러 시장 예상돼
HD현대중공업이 건조한 차세대 이지스 구축함(KDX-III Batch-II) [사진 = HD현대중공업]...

[3] 조선·해양 당기고 전력기기 밀고…실적 견인[HD현대 순항②] (12시간 전)
출판사: 뉴시스
기사 링크: https://www.newsis.com/view/NISX20250207_0003057371
본문 미리보기: 조선·해양 당기고 전력기기 밀고…실적 견인[HD현대 순항②]
등록 2025.02.09 10:01:00수정 2025.02.09 11:20:24
작게
크게
[서울=뉴시스] HD한국조선...

[4] HD현대중공업, 작년 영업익 295% 껑충 ‘마이너스 순차입금’ 전환 (3일 전)
출판사: 블로터
기사 링크: https://www.bloter.net/news/articleView.html?idxno=630889
본문 미리보기: HD현대중공업이 진수한 정조대왕함 /사진 제공=HD현대중공업
  HD현대중공업이 지난해의 호실적에 힘입어 마이너스 순차입금으로 전환했다. 건조 물량이 늘고 선가가 오르면서 매출과 

KeyboardInterrupt: 

In [9]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

# ✅ 웹드라이버 실행 (헤드리스 X)
driver = webdriver.Chrome()

# ✅ 구글 뉴스 목록 첫 페이지 URL
base_url = "https://www.google.com/search?sca_esv=441e37307c978c03&biw=1728&bih=888&q=%ED%98%84%EB%8C%80%EC%A4%91%EA%B3%B5%EC%97%85&tbm=nws&source=lnms"
driver.get(base_url)

# ✅ 페이지 로딩 대기
wait = WebDriverWait(driver, 10)

# ✅ 뉴스 데이터 저장 리스트
news_data = []

for page in range(1, 11):
    print(f"\n🔹 {page}페이지 크롤링 시작...")

    # ✅ 페이지 로딩 대기 (StaleElementReferenceException 방지)
    time.sleep(3)

    # ✅ 뉴스 컨테이너 가져오기
    try:
        news_list = driver.find_elements(By.CLASS_NAME, "SoaBEf")
        print(f"총 {len(news_list)}개의 뉴스를 찾았습니다.")
    except:
        print("❌ 뉴스 요소를 찾을 수 없습니다. 종료합니다.")
        break

    # ✅ 뉴스 목록에서 데이터 추출
    for i, news in enumerate(news_list):
        try:
            title = news.find_element(By.CLASS_NAME, "n0jPhd").text  # 제목
            summary = news.find_element(By.CLASS_NAME, "GI74Re").text  # 요약
            publisher = news.find_element(By.CLASS_NAME, "MgUUmf").text  # 출판사
            date = news.find_element(By.CLASS_NAME, "OSrXXb").text  # 날짜
            article_url = news.find_element(By.CLASS_NAME, "WlydOe").get_attribute("href")  # 기사 링크

            print(f"\n[{(page-1)*10 + i+1}] {title} ({date})")
            print(f"출판사: {publisher}")
            print(f"기사 링크: {article_url}")

            # ✅ 기사 본문 크롤링
            driver.execute_script("window.open('');")  # 새 탭 열기
            driver.switch_to.window(driver.window_handles[1])  # 새 탭으로 전환
            driver.get(article_url)  # 기사 페이지 열기
            time.sleep(3)  # 페이지 로딩 대기

            # ✅ 본문 추출 (itemprop="articleBody" → 실패 시 class="article")
            try:
                article_body = driver.find_element(By.CSS_SELECTOR, '[itemprop="articleBody"]').text
            except:
                try:
                    article_body = driver.find_element(By.CLASS_NAME, "article").text
                except:
                    try:
                        article_body = driver.find_element(By.CLASS_NAME, "article-text").text
                    except:
                        try:
                            article_body = driver.find_element(By.CLASS_NAME, "article_view").text
                        except:
                            article_body = ""

            print(f"📰 본문 미리보기: {article_body[:100]}...")  # 본문 일부 출력

            # ✅ 새 탭 닫고 원래 탭으로 이동
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(3)

            # ✅ 데이터 저장
            news_data.append({
                "제목": title,
                "요약": summary,
                "출판사": publisher,
                "날짜": date,
                "기사 링크": article_url,
                "본문": article_body
            })

        except StaleElementReferenceException:
            print("⚠️ StaleElementReferenceException 발생! 다시 시도합니다.")
            time.sleep(2)
            continue
        except Exception as e:
            print(f"❌ 오류 발생: {e}")

    # ✅ 다음 페이지 버튼 클릭
    try:
        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="botstuff"]/div/div[3]/table/tbody/tr/td[{page+2}]/a')))
        next_button.click()
        time.sleep(3)
        page += 1  # 페이지 카운트 증가
    except NoSuchElementException:
        print("\n✅ 더 이상 다음 페이지가 없습니다. 크롤링 종료.")
        break
    except Exception as e:
        print(f"오류!: {e}")

# ✅ 데이터프레임 생성
df = pd.DataFrame(news_data)

# ✅ CSV 파일로 저장
df.to_csv("google_news_hyundai_all_pages.csv", encoding="utf-8-sig", index=False)
print("\n✅ 데이터 수집 완료! 'google_news_hyundai_all_pages.csv' 파일로 저장되었습니다.")

# ✅ 드라이버 종료
driver.quit()



🔹 1페이지 크롤링 시작...
총 10개의 뉴스를 찾았습니다.

[1] 동서발전, HD현대중공업과 '탄소 에너지 공동 개발'...수소·암모니아 엔진 및 청정수소 기술 개발 협력 (2일 전)
출판사: 글로벌이코노믹
기사 링크: https://www.g-enews.com/article/Real-Estate/2025/02/202502071216274491a9fc143920_1
📰 본문 미리보기: 이미지 확대보기
권명호 한국동서발전 사장(오른쪽)과 한주석 HD현대중공업 엔진기계사업대표가 지난 6일 HD현대중공업 본사에서 업무협약을 체결하고 기념사진을 찍고 있다. 사진=한국동...

[2] [단독] HD현대·한화, K해양방산 ‘원팀’ 합의 (3일 전)
출판사: 매일경제
기사 링크: https://www.mk.co.kr/news/business/11234873
📰 본문 미리보기: 수출마케팅 주도권 분담 ‘큰그림’ 합의
향후 5년간 300억달러 시장 예상돼
HD현대중공업이 건조한 차세대 이지스 구축함(KDX-III Batch-II) [사진 = HD현대중공업]...

[3] 조선·해양 당기고 전력기기 밀고…실적 견인[HD현대 순항②] (14시간 전)
출판사: 뉴시스
기사 링크: https://www.newsis.com/view/NISX20250207_0003057371
📰 본문 미리보기: 조선·해양 당기고 전력기기 밀고…실적 견인[HD현대 순항②]
등록 2025.02.09 10:01:00수정 2025.02.09 11:20:24
작게
크게
[서울=뉴시스] HD한국조선...

[4] HD현대중·한화오션·삼성중, 지난해 대폭 흑자…올해도 ‘순풍’ (3일 전)
출판사: 한겨레
기사 링크: https://www.hani.co.kr/arti/economy/marketing/1181143.html
📰 본문 미리보기: 기사를 읽어드립니다
3:32
에이치디(HD)현대중공업이 지난해 건조해 인도한 174,000 입방미터(㎥)급 엘엔지(LNG) 운반선. 에이치디한국조선해양 제공
국내 조선 3